In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes trl==0.8.6

In [ ]:
import huggingface_hub
import torch
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import gc
import re
import pandas as pd
from tqdm import tqdm
huggingface_hub.login(token="token")

In [ ]:
Q_TKN = '<  question>' #깃허브에 업로드할 때 생략되어 나와서 tab 추가함
C_TKN = '<  context>'
A_TKN = '<  sys>'
EOS = '<|endoftext|>'
SEP = '<|sep|>'
def make_prompt(question, context):
    context = re.sub('[.]{2,5}', '', context)
    prompt = C_TKN + context.replace('.', f".{SEP} ")
    prompt = prompt.replace(f'{SEP} "', '"').replace(f"{SEP} '", "'")
    prompt += f"""{Q_TKN}{question}{SEP} {A_TKN}"""
    return prompt

# 데이터셋 불러오기 및 저장

In [ ]:
df = load_dataset('jaeyong2/RAG-COT-Ko')
df.to_pandas()

In [ ]:
import pandas as pd
q = []
r = []
a = []
for i in tqdm(range(len(df))):
  if len(df['RAW Ground Truth'][i]) == 1:
    q.append(df['Question'][i])
    r.append(df['RAW Ground Truth'][i][0])
    a.append(df['Final Answer'][i])
new = pd.DataFrame({'question': q, 'context':r, 'answer':a})
new

100%|██████████| 376074/376074 [00:05<00:00, 72091.14it/s]


,question,context,answer
0,"작가가 노무현 대통령에 대한 첫 번째 기억은 무엇이고, 두 번째 기억은 어떤 상황에...",나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금 흐른 이후에 한...,작가가 노무현 대통령에 대한 첫 번째 기억은 어릴 때 길거리에 붙은 노무현의 대선 ...
1,조아라 대리가 서핑을 하면서 가장 기억에 남는 추억은 무엇인가요?,"최근에는 국내 양양, 제주, 부산 송정 등 서핑 스팟이 많이 생겨서 즐길 수 있는 ...",조아라 대리가 서핑을 하면서 가장 기억에 남는 추억은 제주도에서 2박 3일간의 나홀...
2,일러전쟁에서 일본이 러시아를 이길 수 있었던 주요 전략가와 그의 전술은 무엇이었는가?,쓰시마 해전에서 일본에는 천재적인 전략가 도고 헤이아치로와 그에게는 독보적인 해전전...,"일러전쟁에서 일본이 러시아를 이길 수 있었던 주요 전략가는 도고 헤이아치로이며, 그..."
3,2021년 2분기 국내 상업용 부동산 시장의 총 투자 규모와 전 분기 대비 증가율은...,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,"2021년 2분기 국내 상업용 부동산 시장의 총 투자 규모는 4조7000억원이며, ..."
4,"엘케이비앤파트너스가 최근 지원한 사회공헌 활동 중 하나는 무엇이며, 이 활동의 목적...",9월 10일 서울 서초구 법무법인 엘케이비앤파트너스 사무실에서 ‘자립준비청년’ 근로...,엘케이비앤파트너스가 최근 지원한 사회공헌 활동 중 하나는 자립을 앞둔 아동양육시설이...
...,...,...,...
280775,"동해시 망상동 삼곡 약천마을에서 감상할 수 있는 시조의 제목과, 이 마을의 위치에 ...",동해시 망상동 삼곡 약천마을은 약천 남구만이 '동창이 밝았느냐'의 시조창작무대를 돌...,동해시 망상동 삼곡 약천마을에서 감상할 수 있는 시조의 제목은 '동창이 밝았느냐'이...
280776,"세 권의 책 중에서, 나이에 따른 차별과 그에 대한 인식 변화를 주제로 다룬 책은 ...",나는 에이지즘에 반대한다작가애슈턴 애플화이트출판시공사발매2016.12.15.평점리뷰...,"세 권의 책 중에서, 나이에 따른 차별과 그에 대한 인식 변화를 주제로 다룬 책은 ..."
280777,스베덴보리가 3년 또는 4년 동안의 부재허가를 얻기 위해 어떤 조건을 약속하였는지 ...,이번에는 3년 또는 4년 동안의 부재허가를 얻었다. 이렇게 오랜 동안의 허가를 얻기...,"스베덴보리가 3년 또는 4년 동안의 부재허가를 얻기 위해, 그는 광산국의 동료들과 ..."
280778,"프라다가 라프 시몬스를 공동 크리에이티브 디렉터로 영입한 배경과 그의 이력, 그리고...",시작은 철옹성 같던 프라다의 라프 시몬스 영입 소식이었다. 1931년 설립 이후 단...,"프라다는 라프 시몬스를 공동 크리에이티브 디렉터로 영입하며, 40년 넘게 프라다를 ..."


In [ ]:
new.to_csv('rag_full.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('rag_full.csv')
for idx, row in df.iterrows():
  df.loc[idx, 'text'] = make_prompt(row['question'], row['context'], row['answer'])
df

,question,context,answer,text
0,"작가가 노무현 대통령에 대한 첫 번째 기억은 무엇이고, 두 번째 기억은 어떤 상황에...",나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금 흐른 이후에 한...,작가가 노무현 대통령에 대한 첫 번째 기억은 어릴 때 길거리에 붙은 노무현의 대선 ...,<context>나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금...
1,조아라 대리가 서핑을 하면서 가장 기억에 남는 추억은 무엇인가요?,"최근에는 국내 양양, 제주, 부산 송정 등 서핑 스팟이 많이 생겨서 즐길 수 있는 ...",조아라 대리가 서핑을 하면서 가장 기억에 남는 추억은 제주도에서 2박 3일간의 나홀...,"<context>최근에는 국내 양양, 제주, 부산 송정 등 서핑 스팟이 많이 생겨서..."
2,일러전쟁에서 일본이 러시아를 이길 수 있었던 주요 전략가와 그의 전술은 무엇이었는가?,쓰시마 해전에서 일본에는 천재적인 전략가 도고 헤이아치로와 그에게는 독보적인 해전전...,"일러전쟁에서 일본이 러시아를 이길 수 있었던 주요 전략가는 도고 헤이아치로이며, 그...",<context>쓰시마 해전에서 일본에는 천재적인 전략가 도고 헤이아치로와 그에게는...
3,2021년 2분기 국내 상업용 부동산 시장의 총 투자 규모와 전 분기 대비 증가율은...,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,"2021년 2분기 국내 상업용 부동산 시장의 총 투자 규모는 4조7000억원이며, ...",<context>CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시...
4,"엘케이비앤파트너스가 최근 지원한 사회공헌 활동 중 하나는 무엇이며, 이 활동의 목적...",9월 10일 서울 서초구 법무법인 엘케이비앤파트너스 사무실에서 ‘자립준비청년’ 근로...,엘케이비앤파트너스가 최근 지원한 사회공헌 활동 중 하나는 자립을 앞둔 아동양육시설이...,<context>9월 10일 서울 서초구 법무법인 엘케이비앤파트너스 사무실에서 ‘자...
...,...,...,...,...
280775,"동해시 망상동 삼곡 약천마을에서 감상할 수 있는 시조의 제목과, 이 마을의 위치에 ...",동해시 망상동 삼곡 약천마을은 약천 남구만이 '동창이 밝았느냐'의 시조창작무대를 돌...,동해시 망상동 삼곡 약천마을에서 감상할 수 있는 시조의 제목은 '동창이 밝았느냐'이...,<context>동해시 망상동 삼곡 약천마을은 약천 남구만이 '동창이 밝았느냐'의 ...
280776,"세 권의 책 중에서, 나이에 따른 차별과 그에 대한 인식 변화를 주제로 다룬 책은 ...",나는 에이지즘에 반대한다작가애슈턴 애플화이트출판시공사발매2016.12.15.평점리뷰...,"세 권의 책 중에서, 나이에 따른 차별과 그에 대한 인식 변화를 주제로 다룬 책은 ...",<context>나는 에이지즘에 반대한다작가애슈턴 애플화이트출판시공사발매2016.<...
280777,스베덴보리가 3년 또는 4년 동안의 부재허가를 얻기 위해 어떤 조건을 약속하였는지 ...,이번에는 3년 또는 4년 동안의 부재허가를 얻었다. 이렇게 오랜 동안의 허가를 얻기...,"스베덴보리가 3년 또는 4년 동안의 부재허가를 얻기 위해, 그는 광산국의 동료들과 ...",<context>이번에는 3년 또는 4년 동안의 부재허가를 얻었다.<|sep|> ...
280778,"프라다가 라프 시몬스를 공동 크리에이티브 디렉터로 영입한 배경과 그의 이력, 그리고...",시작은 철옹성 같던 프라다의 라프 시몬스 영입 소식이었다. 1931년 설립 이후 단...,"프라다는 라프 시몬스를 공동 크리에이티브 디렉터로 영입하며, 40년 넘게 프라다를 ...",<context>시작은 철옹성 같던 프라다의 라프 시몬스 영입 소식이었다.<|sep...


In [ ]:
q = [df['question'][i] for i in range(len(df)) if len(df['text'][i])<1024]
a = [df['answer'][i] for i in range(len(df)) if len(df['text'][i])<1024]
c = [df['context'][i] for i in range(len(df)) if len(df['text'][i])<1024]
t = [df['text'][i] for i in range(len(df)) if len(df['text'][i])<1024]

new = pd.DataFrame({'question': q, 'context':c, 'answer':a, 'text':t})

In [ ]:
new.to_csv('rag_filtering.csv', index=False)

In [ ]:
df = pd.read_csv('rag_filtering.csv')
df

,question,context,answer,text
0,"작가가 노무현 대통령에 대한 첫 번째 기억은 무엇이고, 두 번째 기억은 어떤 상황에...",나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금 흐른 이후에 한...,작가가 노무현 대통령에 대한 첫 번째 기억은 어릴 때 길거리에 붙은 노무현의 대선 ...,<context>나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금...
1,조아라 대리가 서핑을 하면서 가장 기억에 남는 추억은 무엇인가요?,"최근에는 국내 양양, 제주, 부산 송정 등 서핑 스팟이 많이 생겨서 즐길 수 있는 ...",조아라 대리가 서핑을 하면서 가장 기억에 남는 추억은 제주도에서 2박 3일간의 나홀...,"<context>최근에는 국내 양양, 제주, 부산 송정 등 서핑 스팟이 많이 생겨서..."
2,일러전쟁에서 일본이 러시아를 이길 수 있었던 주요 전략가와 그의 전술은 무엇이었는가?,쓰시마 해전에서 일본에는 천재적인 전략가 도고 헤이아치로와 그에게는 독보적인 해전전...,"일러전쟁에서 일본이 러시아를 이길 수 있었던 주요 전략가는 도고 헤이아치로이며, 그...",<context>쓰시마 해전에서 일본에는 천재적인 전략가 도고 헤이아치로와 그에게는...
3,2021년 2분기 국내 상업용 부동산 시장의 총 투자 규모와 전 분기 대비 증가율은...,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,"2021년 2분기 국내 상업용 부동산 시장의 총 투자 규모는 4조7000억원이며, ...",<context>CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시...
4,"엘케이비앤파트너스가 최근 지원한 사회공헌 활동 중 하나는 무엇이며, 이 활동의 목적...",9월 10일 서울 서초구 법무법인 엘케이비앤파트너스 사무실에서 ‘자립준비청년’ 근로...,엘케이비앤파트너스가 최근 지원한 사회공헌 활동 중 하나는 자립을 앞둔 아동양육시설이...,<context>9월 10일 서울 서초구 법무법인 엘케이비앤파트너스 사무실에서 ‘자...
...,...,...,...,...
192783,"코로나19 관련 토론회에서 다루는 주제 중, 인권적 문제점을 중점으로 다루는 세션은...",제1부에서는 「감염병의 예방과 관리에 관한 법률」(약칭 ‘감염병 예방법’)을 중심으...,제1부에서 인권적 문제점을 중점으로 다루는 주제는 감염병 예방법을 중심으로 한 방역...,<context>제1부에서는 「감염병의 예방과 관리에 관한 법률」(약칭 ‘감염병 예...
192784,"동해시 망상동 삼곡 약천마을에서 감상할 수 있는 시조의 제목과, 이 마을의 위치에 ...",동해시 망상동 삼곡 약천마을은 약천 남구만이 '동창이 밝았느냐'의 시조창작무대를 돌...,동해시 망상동 삼곡 약천마을에서 감상할 수 있는 시조의 제목은 '동창이 밝았느냐'이...,<context>동해시 망상동 삼곡 약천마을은 약천 남구만이 '동창이 밝았느냐'의 ...
192785,"세 권의 책 중에서, 나이에 따른 차별과 그에 대한 인식 변화를 주제로 다룬 책은 ...",나는 에이지즘에 반대한다작가애슈턴 애플화이트출판시공사발매2016.12.15.평점리뷰...,"세 권의 책 중에서, 나이에 따른 차별과 그에 대한 인식 변화를 주제로 다룬 책은 ...",<context>나는 에이지즘에 반대한다작가애슈턴 애플화이트출판시공사발매2016.<...
192786,스베덴보리가 3년 또는 4년 동안의 부재허가를 얻기 위해 어떤 조건을 약속하였는지 ...,이번에는 3년 또는 4년 동안의 부재허가를 얻었다. 이렇게 오랜 동안의 허가를 얻기...,"스베덴보리가 3년 또는 4년 동안의 부재허가를 얻기 위해, 그는 광산국의 동료들과 ...",<context>이번에는 3년 또는 4년 동안의 부재허가를 얻었다.<|sep|> ...


In [ ]:
print(df['question'][0])
print(df['answer'][0])
print(df['context'][0])
print(df['text'][0])

작가가 노무현 대통령에 대한 첫 번째 기억은 무엇이고, 두 번째 기억은 어떤 상황에서 발생했는가?
작가가 노무현 대통령에 대한 첫 번째 기억은 어릴 때 길거리에 붙은 노무현의 대선 홍보 현수막을 보고 아빠에게 '아빠, 난 저 사람이 맘에 들어, 인상이 아주 선해.'라고 말한 것이다. 두 번째 기억은 노무현 대통령의 서거 이후 교실에서 노무현 대통령을 기리는 건물을 크게 짓는다는 신문기사를 읽고 벌어진 작은 논쟁이다.
나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금 흐른 이후에 한국에 태어났다. 정치를 모르고 컸다. 단지 어릴 때, 길거리에 붙은 노무현의 대선 홍보 현수막을 보고 아빠한테 '아빠, 난 저 사람이 맘에 들어, 인상이 아주 선해.' 라고 했던 기억은 선명하다. 아빠가 사람 볼 줄 안다고 웃었고, 옆에 어른들도 덩달아 웃었다. 그것이 노무현에 대한 나의 첫 번째 기억이다. 두번째 기억은 노무현 대통령의 서거 이후 교실에서 벌어졌던 작은 논쟁이다. 노무현 대통령을 기리는 무슨 건물을 크게 짓는다는 신문기사를 읽고 있었고, 역사&정치 의식이 부재했던 나는..
<|context|>나는 독재정권 시대 때는 아직 태어나지 않았었고 잔인한 시절이 조금 흐른 이후에 한국에 태어났다.<|sep|>  정치를 모르고 컸다.<|sep|>  단지 어릴 때, 길거리에 붙은 노무현의 대선 홍보 현수막을 보고 아빠한테 '아빠, 난 저 사람이 맘에 들어, 인상이 아주 선해.' 라고 했던 기억은 선명하다.<|sep|>  아빠가 사람 볼 줄 안다고 웃었고, 옆에 어른들도 덩달아 웃었다.<|sep|>  그것이 노무현에 대한 나의 첫 번째 기억이다.<|sep|>  두번째 기억은 노무현 대통령의 서거 이후 교실에서 벌어졌던 작은 논쟁이다.<|sep|>  노무현 대통령을 기리는 무슨 건물을 크게 짓는다는 신문기사를 읽고 있었고, 역사&정치 의식이 부재했던 나는<|question|>작가가 노무현 대통령에 대한 첫 번째 기억은 무엇이고, 두 번째 기억은 어떤 상황에서 발생했

#모델 불러오기 및 학습

In [ ]:
lora_config = LoraConfig(
              r=8,
              lora_alpha=32,
              target_modules=["query_key_value"],
              lora_dropout=0.05,
              bias="none",
              task_type="CAUSAL_LM"
            )
def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    elif peft == 'qlora':
        #양자화 설정 정의
        bnb_config = BitsAndBytesConfig(
                  load_in_4bit=True,
                  bnb_4bit_use_double_quant=True,
                  bnb_4bit_quant_type="nf4",
                  bnb_4bit_compute_dtype=torch.float16
              )
        #모델 불러오기
        model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)
        #lora 설정 적용
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    return model, tokenizer

In [ ]:
def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1,
                          gradient_checkpointing=False,
                          model_id="EleutherAI/polyglot-ko-1.3b",
                          peft=None):

    print(f"배치 사이즈: {batch_size}")
    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)
    #Q_TKN과 A_TKN, C_TKN 추가
    tokenizer.add_tokens(Q_TKN, special_tokens=True)
    tokenizer.add_tokens(A_TKN, special_tokens=True)
    tokenizer.add_tokens(C_TKN, special_tokens=True)
    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False
    dataset = load_dataset('csv', data_files='rag_filtering.csv')['train']
    #print(dataset[0])
    dataset = dataset.train_test_split(test_size=180000, shuffle=True, seed=8)
    dataset = dataset['test'].train_test_split(test_size=18000, shuffle=True, seed=8)
    #print(dataset)
    #인자 설정
    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir="./polyglot-ko-1.3b-RAG-COT-ko",
        num_train_epochs=1
      )
    trainer = SFTTrainer(
          model=model,
          train_dataset=dataset['train'],
          peft_config=lora_config,
          dataset_text_field="text",
          max_seq_length=1024,
          tokenizer=tokenizer,
          args=training_args,
          packing=False,
      )
    try:
      trainer.train()
      repo_id = f"won2133/polyglot-ko-1.3b-RAG-COT-ko"
      trainer.push_to_hub(repo_id)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        del model, dataset
        #메모리 삭제
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
gpu_memory_experiment(batch_size=32, peft='qlora')
torch.cuda.empty_cache()